# Лабораторная работа #2
## По предмету: "Алгоритмические основы мультимедийных технологий"
## Тема: "Наложение фильтров на видео"
### Выполнили: Терентьев Егор Дмитриевич НФИмд-01-23 1132236902 и Юдин Герман Станиславович НФИмд-02-23 1132236901

### Задание:
#### Применить алгоритмы обработки фотографий из 1ой лабораторной работы на видео

Импортируем нужные библиотеки

In [16]:
from matplotlib.image import imread
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2

### Алгоритмы обработки изображений из прошлой лабораторной работы

Здесь представлены алгоритмы из первой лабораторной работы: **Негатив**, **Яркость**, **Блюр**

In [17]:
def negative(matrix):
	new_matrix = []
	for i in range(len(matrix)):
		lst = []
		for j in range(len(matrix[i])):
			lst.append([255 - matrix[i][j][k] for k in range(3)])
		new_matrix.append(lst)
	return new_matrix


def brightness(matrix, k):
	new_matrix = []
	average = 0
	for i in range(len(matrix)):
		lst = []
		for j in range(len(matrix[i])):
			if k > 0:
				lst.append((min(matrix[i][j][0] + k, 255), min(matrix[i][j][1] + k, 255), min(matrix[i][j][2] + k, 255)))
			else:
				lst.append((max(matrix[i][j][0] + k, 0), max(matrix[i][j][1] + k, 0), max(matrix[i][j][2] + k, 0)))
		new_matrix.append(lst)
	return new_matrix


def blur(matrix, n=1, a=0.5):
	new_matrix = []
	for i in range(len(matrix)):
		lst = []
		for j in range(len(matrix[i])):
			count = -1
			summary = [-int(matrix[i][j][k]) for k in range(3)]
			for k in range(max(i - n, 0), min(i + n + 1, len(matrix))):
				for l in range(max(j - n, 0), min(j + n + 1, len(matrix[0]))):
					count += 1
					for p in range(3):
						summary[p] += int(matrix[k][l][p])
			for p in range(3):
				summary[p] = int(summary[p] / count * a)
				summary[p] += int(matrix[i][j][p] * (1 - a))
				summary[p] = min(summary[p], 255)
			
			lst.append(summary)
		new_matrix.append(lst)
	return new_matrix

### Функция для наложения эффектов на входящие видео

***Функция video_modify принимает путь к видеофайлу и выполняет несколько операций над каждым кадром видео. Она считывает видео, разбивает его на три части и применяет различные преобразования к каждой части: увеличение яркости, создание негатива и размытие. Затем измененные кадры записываются в новый видеофайл***

In [18]:
def video_modify(video):
	cap = cv2.VideoCapture(video) #создаем объект для считывания видео
	frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) 
	frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
	frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
	fps = cap.get(cv2.CAP_PROP_FPS)
	
	fourcc = cv2.VideoWriter_fourcc(*'XVID') #кодек для записи видео
	out = cv2.VideoWriter('output.mp4', fourcc, fps, (frameWidth,frameHeight)) #создаем объект для записи видео
	
	buf = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8')) #создаем буфер для кадров
	new_buf = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8')) #создаем буфер для измененных кадров
	
	fc = 0 #счетчик кадров
	ret = True #флаг, показывающий, был ли считан кадр
	
	while (fc < frameCount  and ret): #считываем кадры
		ret, buf[fc] = cap.read() 
		fc += 1
	
	blur_counter = 0
	third_part = len(new_buf) / 3 #разбиваем видео на три части
	for i in range(len(buf)):
		print(f"Frame {i + 1}/{len(new_buf)}")
		if i < third_part: #первая часть - увеличиваем яркость
			new_buf[i] = brightness(buf[i], 255 - (i * 5))
		elif i < third_part * 2: #вторая часть - негатив
			new_buf[i] = negative(buf[i])
		else: #третья часть - размытие
			new_buf[i] = blur(buf[i], 3, blur_counter / third_part)
			blur_counter += 1
	
	for frame in new_buf: #записываем измененные кадры
		out.write(frame)
	
	cap.release()
	out.release()
	
	cv2.destroyAllWindows()

### Функция для отображения получившихся результатов

In [19]:
def compare_videos(original_video, resulting_video):
    # Открываем оригинальное видео
    original_cap = cv2.VideoCapture(original_video)
    original_fps = original_cap.get(cv2.CAP_PROP_FPS)

    # Открываем видео с результатом
    resulting_cap = cv2.VideoCapture(resulting_video)
    resulting_fps = resulting_cap.get(cv2.CAP_PROP_FPS)

    # Создаем окно для отображения видео
    cv2.namedWindow("Comparison", cv2.WINDOW_NORMAL)

    while True:
        # Считываем кадры из видео
        ret1, frame1 = original_cap.read()
        ret2, frame2 = resulting_cap.read()

        # Завершаем цикл, если кадры закончились
        if not ret1 or not ret2:
            break

        # Считаем разницу между кадрами
        #diff = cv2.absdiff(frame1, frame2)

        # отображение видео
        cv2.imshow("Comparison", cv2.hconcat([frame1, frame2]))

        # закрытие окна при нажатии клавиши q
        if cv2.waitKey(int(1000 / max(original_fps, resulting_fps))) & 0xFF == ord('q'):
            break

    # Release the video captures and destroy the window
    original_cap.release()
    resulting_cap.release()
    cv2.destroyAllWindows()


In [20]:
#video_modify("video(10sec).mp4") #запуск функции для изменения видео
compare_videos("video(10sec).mp4", "output.mp4") #сравнение видео 

### Алгоритм перехода между видео

#### Описание и реализация алгоритма слайд
Для реализация нам требуется transition_frames - это количество кадров, которые будут использоваться для перехода. Оно равно количеству кадров в более коротком из двух видео.
В цикле для каждого кадра перехода (i от 0 до transition_frames):
Создается пустой кадр transition_frame с теми же размерами и типом данных, что и исходные кадры.
Левая часть transition_frame заполняется соответствующими пикселями из первого видео (buf1). Ширина этой части увеличивается с каждым новым кадром перехода.
Правая часть transition_frame заполняется соответствующими пикселями из второго видео (buf2). Ширина этой части уменьшается с каждым новым кадром перехода.
Затем каждый кадр записывается последовательно

In [21]:
def transition(video1, video2):
	cap1 = cv2.VideoCapture(video1)
	cap2 = cv2.VideoCapture(video2)
	frameCount1 = int(cap1.get(cv2.CAP_PROP_FRAME_COUNT))
	frameCount2 = int(cap2.get(cv2.CAP_PROP_FRAME_COUNT))
	frameWidth = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
	frameHeight = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
	fps = cap1.get(cv2.CAP_PROP_FPS)
	
	fourcc = cv2.VideoWriter_fourcc(*'XVID')
	out = cv2.VideoWriter('transition_output.mp4', fourcc, fps, (frameWidth, frameHeight))
	
	buf1 = np.empty((frameCount1, frameHeight, frameWidth, 3), np.dtype('uint8'))
	buf2 = np.empty((frameCount2, frameHeight, frameWidth, 3), np.dtype('uint8'))
	
	fc1 = 0
	ret1 = True
	while (fc1 < frameCount1 and ret1):
		ret1, buf1[fc1] = cap1.read()
		fc1 += 1

	fc2 = 0
	ret2 = True
	while (fc2 < frameCount2 and ret2):
		ret2, buf2[fc2] = cap2.read()
		fc2 += 1

	transition_frames = min(frameCount1, frameCount2)
	for i in range(transition_frames):
		print(f"Frame {i + 1}/{transition_frames}")
		transition_frame = np.zeros((frameHeight, frameWidth, 3), np.uint8)
		transition_frame[:, :int(frameWidth * (i / transition_frames))] = buf1[i][:, :int(frameWidth * (i / transition_frames))]
		transition_frame[:, int(frameWidth * (i / transition_frames)):] = buf2[i][:, int(frameWidth * (i / transition_frames)):]

		out.write(transition_frame)

	cap1.release()
	cap2.release()
	out.release()

	cv2.destroyAllWindows()

In [22]:
def display_video(path):
    cap = cv2.VideoCapture(path)  # Открываем видеофайл
    while cap.isOpened():
        ret, frame = cap.read()  # Считываем кадр из видео
        if not ret:
            break
        cv2.imshow('Video', frame)  # Отображаем кадр
        if cv2.waitKey(1) & 0xFF == ord('q'):  # Ждем нажатия клавиши "q"
            break
    cap.release()  # Закрываем видеофайл
    cv2.destroyAllWindows()  # Закрываем окно отображения видео

#transition("video(10sec).mp4", "video2.mp4")
display_video("transition_output.mp4")

### Заключение
После выполнения данной лабораторной работы мы научились реализовывать и применять различные фильтры на видео